# Clase 03 - Datos multidimensionales, de jerarquía y de flujo

Profesor: **Fernando Becerra**, f.becerra@udd.cl, [www.fernandobecerra.com](www.fernandobecerra.com)

Esta semana expanderemos más nuestro conocimiento de distintos tipos de gráficos para los casos de datos multidimensionales, de jerarquía y de flujo.

Comencemos importando lo mismo de siempre.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

## Datos multidimensionales

Cuando tenemos datos que tienen más de 3 ó 4 propiedades, gráficos como el de barra o el de dispersión quedan cortos y no son suficiente, es por eso que usamos otras técnicas de representación para ver todas las variables al mismo tiempo y explorar si hay alguna relación entre ellas.

Las coordenadas paralelas y la matriz de diagramas de dispersión las importamos desde `pandas`.

In [ ]:
from pandas.plotting import parallel_coordinates, scatter_matrix

### Coordenadas paralelas

Cargamos datos estándar desde `seaborn`

In [ ]:
data = sns.load_dataset('iris')
data

Y ocupamos las coordenadas paralelas de `pandas`

In [ ]:
ax = parallel_coordinates(data, 'species')

No es el mejor gráfico, pero se ve bien. Probemos con otra serie de datos

In [ ]:
data = pd.read_csv("../../datos/exoplanets.csv")
data.head()

Filtremos sólo las columnas que nos interesan

In [ ]:
df = data[['MASS', 'R', 'BMV', 'MSTAR', 'RSTAR', 'TEFF', 'A', 'PER', 'ECC', 'DIST', 'PLANETDISCMETH']].dropna().reset_index(drop=True)
df.head()

Y probemos las coordenada paralelas

In [ ]:
ax = parallel_coordinates(df, 'PLANETDISCMETH')

Lamentablemente este método tiene varios problemas. Entre otros, el principal es que no se pueden ajustar las escalas del eje y para las distintas columnas. Para resolver eso, usaremos una función especial para crear coordenadas paralelas sacada de [acá](http://benalexkeen.com/parallel-coordinates-in-matplotlib/).

In [ ]:
from matplotlib import ticker

def plot_parallel_coordinates(df, cat_col, cols, colours):
    
    x = [i for i, _ in enumerate(cols)]

    # Create (X-1) sublots along x axis
    fig, axes = plt.subplots(1, len(x)-1, sharey=False, figsize=(15,5))

    # Get min, max and range for each column
    # Normalize the data for each column
    min_max_range = {}
    for col in cols:
        min_max_range[col] = [df[col].min(), df[col].max(), np.ptp(df[col])]
        df[col] = np.true_divide(df[col] - df[col].min(), np.ptp(df[col]))

    # Plot each row
    for i, ax in enumerate(axes):
        for idx in df.index:
            mpg_category = df.loc[idx, cat_col]
            ax.plot(x, df.loc[idx, cols], colours[mpg_category])
        ax.set_xlim([x[i], x[i+1]])

    # Set the tick positions and labels on y axis for each plot
    # Tick positions based on normalised data
    # Tick labels are based on original data
    def set_ticks_for_axis(dim, ax, ticks):
        min_val, max_val, val_range = min_max_range[cols[dim]]
        step = val_range / float(ticks-1)
        tick_labels = [round(min_val + step * i, 2) for i in range(ticks)]
        norm_min = df[cols[dim]].min()
        norm_range = np.ptp(df[cols[dim]])
        norm_step = norm_range / float(ticks-1)
        ticks = [round(norm_min + norm_step * i, 2) for i in range(ticks)]
        ax.yaxis.set_ticks(ticks)
        ax.set_yticklabels(tick_labels)

    for dim, ax in enumerate(axes):
        ax.xaxis.set_major_locator(ticker.FixedLocator([dim]))
        set_ticks_for_axis(dim, ax, ticks=6)
        ax.set_xticklabels([cols[dim]])


    # Move the final axis' ticks to the right-hand side
    ax = plt.twinx(axes[-1])
    dim = len(axes)
    ax.xaxis.set_major_locator(ticker.FixedLocator([x[-2], x[-1]]))
    set_ticks_for_axis(dim, ax, ticks=6)
    ax.set_xticklabels([cols[-2], cols[-1]])


    # Remove space between subplots
    plt.subplots_adjust(wspace=0)

    # Add legend to plot
    plt.legend(
        [plt.Line2D((0,1),(0,0), color=colours[cat]) for cat in df[cat_col].cat.categories],
        df[cat_col].cat.categories,
        bbox_to_anchor=(1.2, 1), loc=2, borderaxespad=0.)

    return fig, ax

Ahora definimos las variables que nos pide la función, entre ellas: el dataframe, la columna que ocuparemos como categoría, las columnas que queremos graficas, y los colores de las categorías en forma de diccionario.

In [ ]:
cat_col = 'PLANETDISCMETH'

cols = ['MASS', 'R', 'BMV', 'MSTAR', 'RSTAR', 'TEFF', 'A', 'PER', 'ECC', 'DIST']
colours = ['#2e8ad8', '#cd3785']

df[cat_col] = df[cat_col].astype('category')

colours = {
    df[cat_col].cat.categories[i]: colours[i] for i, _ in enumerate(df[cat_col].cat.categories)
}


Y ocupamos la función ya definida

In [ ]:
fig, ax = plot_parallel_coordinates(df, 'PLANETDISCMETH', cols, colours)

Como nos devuelve la `fig` y el `ax`, podemos seguir haciéndole otras modificaciones como agregar título y otras.

### Matrices

Otra forma de graficar datos multidimensionales es crear una matriz de relaciones entre todas las variables a ocupar. Para esto, ocuparemos la función `PairGrid` de `seaborn`.

In [ ]:
g = sns.PairGrid(df, hue=cat_col)
plt.show()

Y podemos ir agregando nuestro gráficos en la parte inferior.

In [ ]:
g = sns.PairGrid(df, hue=cat_col)
g.map_lower(sns.scatterplot)
plt.show()

En la diagonal

In [ ]:
g = sns.PairGrid(df, hue=cat_col)
g.map_lower(sns.scatterplot)
g.map_diag(sns.kdeplot, lw=3, legend=False)
plt.show()

Y en la parte superior.

In [ ]:
g = sns.PairGrid(df, hue=cat_col)
g.map_lower(sns.scatterplot)
g.map_diag(sns.kdeplot, lw=3, legend=False)
g.map_upper(sns.kdeplot)
plt.show()

## Jerarquía

Otros tipos de datos con los que nos encontramos frecuentemente, son aquellos en los que hay algún tipo de jerarquía. Por ejemplo, grupo de grupos. Dos visualizaciones son muy comunes para ese tipo de datos: los treemaps y los círculos.

### Treemap

Para hacer un treemap necesitamos instalar el paquete `squarify`

In [ ]:
import squarify

Los datos que cargaremos son los aportes a las campañas del plebiscito de este fin de semana

In [ ]:
df = pd.read_excel('../../datos/aportes_gastos_plebiscito.xlsx', sheet_name=['Aportes', 'Propaganda por Medios Digitales', 'Medios'])
df['Aportes'].head()

Necesitamos procesarlos un poco para que queden en un formato que nos convenga graficarlo. Para eso, calcularemos el aporte total por cada organización en la base de datos y la ordenaremos por el monto del aporte.

In [ ]:
aportes = (
    df['Aportes'].groupby(['Nombre Organización', 'Opcion'])
        .agg(aporte=("Monto ($)", "sum"))
        .sort_values(['Opcion', 'aporte'], ascending=False)
        .reset_index()
)
aportes

Ocupamos `squarify` para crear nuestro primer treemap

In [ ]:
squarify.plot(sizes=aportes['aporte'], alpha=.8 )
plt.axis('off')
plt.show()

Necesitamos mejorarlo, para lo cual agregaremos colores y anotaciones. Tanto los colores como las anotaciones estarán dados por la fracción del aporte total correspondiente a cada opción. Para eso, primero necesitamos calcular el aporte total de cada opción, y calcular la fracción para cada organización.

In [ ]:
totA = aportes[aportes['Opcion'] == 'APRUEBO']['aporte'].sum()
totR = aportes[aportes['Opcion'] == 'RECHAZO']['aporte'].sum()
aportes['porcentaje'] = aportes.apply(
    lambda x: x['aporte']/totA if x['Opcion'] == 'APRUEBO' else x['aporte']/totR,
    axis=1
)
aportes.head()

Con esos datos, calcularmos el color (en una escala de 0 a 1) para cada fila de datos.

In [ ]:
pmin = aportes[aportes['Opcion'] == 'APRUEBO']['porcentaje'].min()
pmax = aportes[aportes['Opcion'] == 'APRUEBO']['porcentaje'].max()
normA = mpl.colors.Normalize(vmin=pmin, vmax=pmax)
cmapA = mpl.cm.Blues

pmin = aportes[aportes['Opcion'] == 'RECHAZO']['porcentaje'].min()
pmax = aportes[aportes['Opcion'] == 'RECHAZO']['porcentaje'].max()
normR = mpl.colors.Normalize(vmin=pmin, vmax=pmax)
cmapR = mpl.cm.Reds

aportes['norm'] = aportes.apply(
    lambda x: normA(x['porcentaje']) if x['Opcion'] == 'APRUEBO' else normR(x['porcentaje']),
    axis=1
)
aportes['color'] = aportes.apply(
    lambda x: cmapA(x['norm']) if x['Opcion'] == 'APRUEBO' else cmapR(x['norm']),
    axis=1
)
aportes.head()

Ahora calculamos las anotaciones que incluiremos, basándonos en un valor de corte que llamaremos `threshold`.

In [ ]:
threshold = 0.5
aportes['nombre'] = aportes.apply(
    lambda x: x['Nombre Organización'].split("-")[0] if x['norm'] > threshold else '',
    axis=1
)


Finalmente graficamos todo.

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(18,8))

squarify.plot(sizes=aportes['aporte'],
              color=aportes['color'],
              label=aportes['nombre'],
              ax=ax)
plt.axis('off')
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(18,8))

squarify.plot(sizes=aportes['aporte'],
              color=aportes['color'],
              label=aportes['nombre'],
              alpha=.8,
              text_kwargs={
                  'fontsize':12,
                  'color': 'white',
                  'weight': 'bold'
              },
              ax=ax,
              edgecolor='#d2d2d2')
plt.axis('off')
plt.show()

### Círculos

Algo parecido podemos hacer pero ahora con círculos. Para ello usaremos datos de población mundial, en format json.

In [ ]:
data = [{'id': 'World', 'datum': 6964195249, 'children' : [
              {'id' : "North America", 'datum': 450448697,
                   'children' : [
                     {'id' : "United States", 'datum' : 308865000},
                     {'id' : "Mexico", 'datum' : 107550697},
                     {'id' : "Canada", 'datum' : 34033000} 
                   ]},
              {'id' : "South America", 'datum' : 278095425, 
                   'children' : [
                     {'id' : "Brazil", 'datum' : 192612000},
                     {'id' : "Colombia", 'datum' : 45349000},
                     {'id' : "Argentina", 'datum' : 40134425}
                   ]},
              {'id' : "Europe", 'datum' : 209246682,  
                   'children' : [
                     {'id' : "Germany", 'datum' : 81757600},
                     {'id' : "France", 'datum' : 65447374},
                     {'id' : "United Kingdom", 'datum' : 62041708}
                   ]},
              {'id' : "Africa", 'datum' : 311929000,  
                   'children' : [
                     {'id' : "Nigeria", 'datum' : 154729000},
                     {'id' : "Ethiopia", 'datum' : 79221000},
                     {'id' : "Egypt", 'datum' : 77979000}
                   ]},
              {'id' : "Asia", 'datum' : 2745929500,  
                   'children' : [
                     {'id' : "China", 'datum' : 1336335000},
                     {'id' : "India", 'datum' : 1178225000},
                     {'id' : "Indonesia", 'datum' : 231369500}
                   ]}
    ]}]

Necesitamos instalar el paquete `circlify`

In [ ]:
import circlify

Y lo ocupamos para calcular los círculos

In [ ]:
# Compute circle positions thanks to the circlify() function
circles = circlify.circlify(
    data, 
    show_enclosure=False, 
    target_enclosure=circlify.Circle(x=0, y=0, r=1)
)
circles

Una vez que tenemos todos los datos de los círculos los graficamos.

In [ ]:
fig, ax = plt.subplots(figsize=(14,14))

# Title
ax.set_title('Repartition of the world population')

# Remove axes
ax.axis('off')

# Find axis boundaries
lim = max(
    max(
        abs(circle.x) + circle.r,
        abs(circle.y) + circle.r,
    )
    for circle in circles
)
plt.xlim(-lim, lim)
plt.ylim(-lim, lim)

# Print circle the highest level (continents):
for circle in circles:
    x, y, r = circle
    ax.add_patch( plt.Circle((x, y), r, alpha=0.5, linewidth=2, color="lightblue"))

In [ ]:
fig, ax = plt.subplots(figsize=(14,14))

# Title
ax.set_title('Repartition of the world population')

# Remove axes
ax.axis('off')

# Find axis boundaries
lim = max(
    max(
        abs(circle.x) + circle.r,
        abs(circle.y) + circle.r,
    )
    for circle in circles
)
plt.xlim(-lim, lim)
plt.ylim(-lim, lim)

# Print circle the highest level (continents):
for circle in circles:
    if circle.level != 2:
        continue
    x, y, r = circle
    ax.add_patch( plt.Circle((x, y), r, alpha=0.5, linewidth=2, color="lightblue"))

# Print circle and labels for the highest level:
for circle in circles:
    if circle.level != 3:
        continue
    x, y, r = circle
    label = circle.ex["id"]
    ax.add_patch( plt.Circle((x, y), r, alpha=0.5, linewidth=2, color="#69b3a2"))
    plt.annotate(label, (x,y ), ha='center', color="white")

# Print labels for the continents
for circle in circles:
    if circle.level != 2:
        continue
    x, y, r = circle
    label = circle.ex["id"]
    plt.annotate(label, (x,y) ,va='center', ha='center', bbox=dict(facecolor='white', edgecolor='black', boxstyle='round', pad=.5))

## Flujos

### Streamgraph

El streamgraph consiste básicamente en un stacked area chart suavizado. Para la parte de suavizar el gráfico será necesatio ocupar `scipy`.

In [ ]:
from scipy import stats

Partamos creando datos aleatorios

In [ ]:
x = np.arange(1990, 2020) # (N,) array-like
y = [np.random.randint(0, 5, size=30) for _ in range(5)] # (M, N) array-like

Y creemos un stacked area chart

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
ax.stackplot(x, y);

Para un streamgraph es necesario mover el gráfico de tal forma que el eje x se ubique a la mitad del gráfico.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
ax.stackplot(x, y, baseline="sym")
ax.axhline(0, color="black", ls="--");

Para suavizar la curva ocuparemos una función `stats.norm.pdf` que se ocupa de la siguiente manera:

In [ ]:
grid = np.linspace(-3, 3, num=100)
plt.plot(grid, stats.norm.pdf(grid));

Entonces podemos definir una función que nos calcula la versión suavizada de cualquier serie de datos.

In [ ]:
def gaussian_smooth(x, y, sd):
    weights = np.array([stats.norm.pdf(x, m, sd) for m in x])
    weights = weights / weights.sum(1)
    return (weights * y).sum(1)

Y ocupamos eso para suavizar nuestros datos iniciales

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
y_smoothed = [gaussian_smooth(x, y_, 1) for y_ in y]
ax.stackplot(x, y_smoothed, baseline="sym");

No está tan suave aún, por lo que ampliamos la función para recibir cualquier grilla (resolución) y así lograr una mejor curva.

In [ ]:
def gaussian_smooth(x, y, grid, sd):
    weights = np.transpose([stats.norm.pdf(grid, m, sd) for m in x])
    weights = weights / weights.sum(0)
    return (weights * y).sum(1)

Y ahora la aplicamos a nuestros datos

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
grid = np.linspace(1985, 2025, num=500)
y_smoothed = [gaussian_smooth(x, y_, grid, 1) for y_ in y]
ax.stackplot(grid, y_smoothed, baseline="sym");

Podemos comparar cómo se ven dos curvas con distinta resolución para tener una idea de como funcionan esos parámetros

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
# sd of 0.6
y_smoothed_1 = [gaussian_smooth(x, y_, grid, 0.6) for y_ in y]
# sd of 1.5
y_smoothed_2 = [gaussian_smooth(x, y_, grid, 1.5) for y_ in y]

ax[0].stackplot(grid, y_smoothed_1, baseline="sym")
ax[1].stackplot(grid, y_smoothed_2, baseline="sym");

### Diagrama de Sankey

Por último, veremos cómo hacer un diagrama de Sankey. Para esto, necesitamos instalar el paquete `pysankey`.

In [ ]:
from pySankey.sankey import sankey

Cargamos unos de los datos que vienen de ejemplo

In [ ]:
url = "https://raw.githubusercontent.com/anazalea/pySankey/master/pysankey/fruits.txt"
df = pd.read_csv(url, sep=" ", names=["true", "predicted"])
df.head()

Y los graficamos

In [ ]:
colors = {
    "apple": "#f71b1b",
    "blueberry": "#1b7ef7",
    "banana": "#f3f71b",
    "lime": "#12e23f",
    "orange": "#f78c1b"
}

sankey(df["true"], df["predicted"], aspect=20, colorDict=colors, fontsize=12)

Podemos agregarle pesos. Para eso carguemos una nueva serie de datos.

In [ ]:
url = "../../datos/customers-goods.csv"
df = pd.read_csv(url, sep=",")
df.head()

Y usemos los pesos para ambos lados del diagrama.

In [ ]:
sankey(
    left=df["customer"], right=df["good"], 
    leftWeight= df["revenue"], rightWeight=df["revenue"], 
    aspect=20, fontsize=20
)

Un muy buen recurso para ver estos tipos de gráficos y más es [la galería de gráficos de python](https://www.python-graph-gallery.com)